# 디렉토리 설정 및 이미지데이터 전처리

In [1]:
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
!pwd

/content/drive/MyDrive/Colab Notebooks


In [5]:
base_dir = './'
categories = ['pedestrian', 'sitter', 'taxier']
nb_classes = len(categories)

In [6]:
# 이미지 크기 지정
image_w = 500
image_h = 500
pixels = image_w*image_h*3

In [7]:
# 이미지데이터 읽어들이기
X = []
Y = []
for idx, category in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    # 이미지
    image_dir = base_dir+category+'/*'
    files = glob.glob(image_dir)
    
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
    
X = np.array(X)
Y = np.array(Y)

# 학습 전용 데이터와 테스트 전용 데이터로 구분
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [8]:
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (303, 500, 500, 3)
X_test shape: (102, 500, 500, 3)


# 일반 CNN 모델 적용

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), (1, 1), input_shape=X_train.shape[1:], padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# 전결합층
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 500, 500, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 250, 250, 32)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 250, 250, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 250, 250, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 125, 125, 32)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 125, 125, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 64)     

In [15]:
# 모델 구축하기
from tensorflow.keras import metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.CategoricalAccuracy()])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
mc = ModelCheckpoint(base_dir+'CNN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
histroy = model.fit(X_train, y_train, batch_size=16, epochs=100, callbacks=[es, mc], validation_split=0.15)

Epoch 1/100
17/17 [==============================] - 82s 5s/step - loss: 19.0751 - categorical_accuracy: 0.3615 - val_loss: 1.0989 - val_categorical_accuracy: 0.2826
Epoch 2/100
17/17 [==============================] - 81s 5s/step - loss: 1.0797 - categorical_accuracy: 0.3678 - val_loss: 1.0991 - val_categorical_accuracy: 0.2826
Epoch 3/100
17/17 [==============================] - 79s 5s/step - loss: 1.0784 - categorical_accuracy: 0.3226 - val_loss: 1.0995 - val_categorical_accuracy: 0.2826
Epoch 4/100
17/17 [==============================] - 79s 5s/step - loss: 1.0826 - categorical_accuracy: 0.3802 - val_loss: 1.0996 - val_categorical_accuracy: 0.2826
Epoch 5/100
17/17 [==============================] - 79s 5s/step - loss: 1.0497 - categorical_accuracy: 0.4552 - val_loss: 1.0945 - val_categorical_accuracy: 0.3043
Epoch 6/100
17/17 [==============================] - 80s 5s/step - loss: 0.9102 - categorical_accuracy: 0.5990 - val_loss: 0.9807 - val_categorical_accuracy: 0.5435
Epoch 7/1

In [17]:
# 모델 평가하기 
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

4/4 [==============================] - 8s 2s/step - loss: 0.3718 - categorical_accuracy: 0.8431
loss= 0.37181589007377625
accuracy= 0.843137264251709


In [18]:
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')

for category in categories:
    for i in range(1,4):
        test_image = base_dir+category+'_test_'+str(i)+'.jpg'
        
        # 이미지 resize
        img = Image.open(test_image)
        img = img.convert("RGB")
        img = img.resize((500,500))
        data = np.asarray(img)
        X = np.array(data)
        X = X.astype("float") / 256
        X = X.reshape(-1, 500,500,3)
        
        # 예측
        pred = model.predict(X)
        print(pred)
        result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
        print(test_image, ':',categories[result[0]])
        print()

[[9.9503577e-01 4.6728225e-03 2.9147544e-04]]
./pedestrian_test_1.jpg : pedestrian

[[9.920399e-01 7.328899e-03 6.311176e-04]]
./pedestrian_test_2.jpg : pedestrian

[[0.82671595 0.0923548  0.08092932]]
./pedestrian_test_3.jpg : pedestrian

[[0.60183364 0.05594093 0.34222537]]
./sitter_test_1.jpg : pedestrian

[[0.6774644  0.18513665 0.13739893]]
./sitter_test_2.jpg : pedestrian

[[0.0036025  0.5966918  0.39970574]]
./sitter_test_3.jpg : sitter

[[0.00102873 0.30185717 0.6971141 ]]
./taxier_test_1.jpg : taxier

[[0.00708231 0.02131307 0.97160465]]
./taxier_test_2.jpg : taxier

[[0.00342804 0.04862165 0.94795036]]
./taxier_test_3.jpg : taxier

